In [9]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.graph import StateGraph, MessagesState, START
from langgraph.prebuilt import ToolNode, tools_condition
from langchain.chat_models import init_chat_model

async def process_query(query: str, server_configs=None):
    """
    Process a query using the MCP client and LangGraph
    
    Args:
        query: The user query to process
        server_configs: Dictionary of server configurations, defaults to math and transparencia servers
        
    Returns:
        The response from the graph
    """
    if server_configs is None:
        server_configs = {
            "math": {
                "command": "python",
                "args": ["./math_server.py"],
                "transport": "stdio",
            },
            "transparencia": {
                "command": "python",
                "args": ["./mcp_server_transparencia.py"],
                "transport": "stdio",
            }
        }
    
    # Initialize the chat model
    model = init_chat_model("openai:gpt-4o-mini")
    
    # Create a fresh client and graph for each query
    async with MultiServerMCPClient(server_configs) as client:
        tools = client.get_tools()
        
        def call_model(state: MessagesState):
            response = model.bind_tools(tools).invoke(state["messages"])
            return {"messages": response}
        
        # Build the graph
        builder = StateGraph(MessagesState)
        builder.add_node(call_model)
        builder.add_node(ToolNode(tools))
        builder.add_edge(START, "call_model")
        builder.add_conditional_edges(
            "call_model", 
            tools_condition,
        )
        builder.add_edge("tools", "call_model")
        graph = builder.compile()
        
        # Process the query
        response = await graph.ainvoke({"messages": query})
        return response


In [10]:
transparencia_response = await process_query("Cuantos contratos tiene el proveedor con ruc 20500000001?")
transparencia_response


{'messages': [HumanMessage(content='Cuantos contratos tiene el proveedor con ruc 20500000001?', additional_kwargs={}, response_metadata={}, id='e27ee8f3-3f48-4d1a-ac03-df2613dc1ed5'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_x7whewpIIQUR7lOM8GWM5N0W', 'function': {'arguments': '{"ruc":"20500000001"}', 'name': 'buscar_contratos_por_ruc'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 165, 'total_tokens': 189, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BV7RXg9syGt4W4I1jyMjCnBAEqbeL', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--9f2ba7d2-de5c-4b55-98b3-16b57c5cd75f-0', tool_calls=[{'name': 'busca

In [11]:
transparencia_response = await process_query("Puedes darme la asistencia del congresista Soto ")
transparencia_response

{'messages': [HumanMessage(content='Puedes darme la asistencia del congresista Soto ', additional_kwargs={}, response_metadata={}, id='1af9f3f5-e349-42d4-b50a-5cdaae5569c6'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_YOJbQoMjmLvepX3js1zVRr0e', 'function': {'arguments': '{"nombre":"Soto"}', 'name': 'obtener_asistencias_congresista'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 160, 'total_tokens': 181, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BV7RZVePVemYD1WgM5lIUtregu8Bv', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--0ecdc00d-54b4-4cc4-a6fd-b21a8427d3e9-0', tool_calls=[{'name': 'obtener_asi

# Forma 2

In [12]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.graph import StateGraph, MessagesState, START
from langgraph.prebuilt import ToolNode, tools_condition
from langchain.chat_models import init_chat_model
import asyncio

class MCPQueryProcessor:
    """
    Clase para procesar consultas utilizando MultiServerMCPClient de manera eficiente.
    Mantiene el cliente y el grafo activos para múltiples consultas.
    """
    
    def __init__(self, server_configs=None):
        """
        Inicializa el procesador de consultas MCP
        
        Args:
            server_configs: Diccionario de configuraciones de servidores
        """
        if server_configs is None:
            self.server_configs = {
                "math": {
                    "command": "python", 
                    "args": ["./math_server.py"], 
                    "transport": "stdio",
                },
                "transparencia": {
                    "command": "python", 
                    "args": ["./mcp_server_transparencia.py"], 
                    "transport": "stdio",
                }
            }
        else:
            self.server_configs = server_configs
            
        self.client = None
        self.graph = None
        self.model = init_chat_model("openai:gpt-4.1")
        
    async def __aenter__(self):
        """Método para iniciar el cliente y construir el grafo"""
        self.client = MultiServerMCPClient(self.server_configs)
        await self.client.__aenter__()
        
        # Obtener herramientas del cliente
        tools = self.client.get_tools()
        
        # Definir la función de llamada al modelo
        def call_model(state: MessagesState):
            response = self.model.bind_tools(tools).invoke(state["messages"])
            return {"messages": response}
        
        # Construir el grafo
        builder = StateGraph(MessagesState)
        builder.add_node(call_model)
        builder.add_node(ToolNode(tools))
        builder.add_edge(START, "call_model")
        builder.add_conditional_edges(
            "call_model", 
            tools_condition,
        )
        builder.add_edge("tools", "call_model")
        self.graph = builder.compile()
        
        return self
        
    async def __aexit__(self, exc_type, exc_val, exc_tb):
        """Cerrar el cliente al salir"""
        if self.client:
            await self.client.__aexit__(exc_type, exc_val, exc_tb)
            
    async def process_query(self, query: str):
        """
        Procesar una consulta usando el grafo existente
        
        Args:
            query: La consulta del usuario
        
        Returns:
            La respuesta del grafo
        """
        if not self.graph:
            raise RuntimeError("El procesador no está inicializado. Usa 'async with' primero.")
            
        response = await self.graph.ainvoke({"messages": query})
        return response

In [16]:
async with MCPQueryProcessor() as processor:
    transparencia_response = await processor.process_query("Cuantos contratos tiene el proveedor con ruc 20500000001?")

transparencia_response

{'messages': [HumanMessage(content='Cuantos contratos tiene el proveedor con ruc 20500000001?', additional_kwargs={}, response_metadata={}, id='9ba55fa0-fdd6-4cff-8659-4ebc8a526bbb'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_twJp0HIn3XRv73eryMRUz202', 'function': {'arguments': '{"ruc":"20500000001"}', 'name': 'buscar_contratos_por_ruc'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 165, 'total_tokens': 189, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_a1102cf978', 'id': 'chatcmpl-BV7WBhZ9ld8GIVarFTC7Af1KhlPYC', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--d5fb946a-4293-40e4-b171-a21a531fdb3b-0', tool_calls=[{'name': 'buscar_co